# 동적 양자화

동적 양자화(dynamic quantization)를 활용하여, LSTM과 유사한 형태의 순환 신경망이 좀더 빠르게 추론하도록 만드는 방법이 있댄다!

이 방법은 모델에서 사용하는 가중치의 규모를 줄이고 수행 속도를 빠르게 만든다.

## 도입

양자화는 모델의 가중치를 정수로 변환하는 과정이다. 이는 모델의 가중치를 훨씬 작은 크기로 표현할 수 있게 해주며, 이는 모델의 메모리 사용량을 줄이고, 모델의 추론 속도를 빠르게 만든다.

## 동적 양자화란 무엇인가?

신경망을 양자화한다는 말의 의미는 가중치나 활성화 함수에서 정밀도가 낮은 정수 표현을 사용하도록 바꾼다는 것이다. 이를 통해 모델의 규모를 줄일 수 있다. 

부동소수점 실수 값을 정수로 바꾸는 것은 본질적으로 실수 값에 어떤 배울을 곱하여 그 결과값을 정수로 반올림 하는 것과 같다. 이 배율을 어떻게 정할 것이냐에 따라 양자화하는 방법도 여러가지로 나뉜다.

핵심은 모델을 수행할 때 데이터의 범위를 보고 적절한 배율을 선택하는 것이다.

In [2]:
import torch
import torch.quantization
import torch.nn as nn
import copy
import os
import time

In [12]:
# define a very, very simple LSTM for demonstration purposes
# in this case, we are wrapping ``nn.LSTM``, one layer, no preprocessing or postprocessing
# inspired by
# `Sequence Models and Long Short-Term Memory Networks tutorial <https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html`_, by Robert Guthrie
# and `Dynamic Quanitzation tutorial <https://pytorch.org/tutorials/advanced/dynamic_quantization_tutorial.html>`__.
class lstm_for_demonstration(nn.Module):
  """Elementary Long Short Term Memory style model which simply wraps ``nn.LSTM``
     Not to be used for anything other than demonstration.
  """
  def __init__(self,in_dim,out_dim,depth):
     super(lstm_for_demonstration,self).__init__()
     self.lstm = nn.LSTM(in_dim,out_dim,depth)

  def forward(self,inputs,hidden):
     out,hidden = self.lstm(inputs,hidden)
     return out, hidden


torch.manual_seed(29592)  # set the seed for reproducibility

#shape parameters
model_dimension=8
sequence_length=20
batch_size=1
lstm_depth=1

# random data for input
inputs = torch.randn(sequence_length,batch_size,model_dimension)
# hidden is actually is a tuple of the initial hidden state and the initial cell state
hidden = (torch.randn(lstm_depth,batch_size,model_dimension), torch.randn(lstm_depth,batch_size,model_dimension))

In [14]:
 # here is our floating point instance
float_lstm = lstm_for_demonstration(model_dimension, model_dimension,lstm_depth)

# this is the call that does the work
quantized_lstm = torch.quantization.quantize_dynamic(
    float_lstm, {nn.LSTM, nn.Linear}, dtype=torch.qint8
)

# show the changes that were made
print('Here is the floating point version of this module:')
print(float_lstm)
print('')
print('and now the quantized version:')
print(quantized_lstm)

RuntimeError: Didn't find engine for operation quantized::linear_prepack NoQEngine

fixme: https://pytorch.org/tutorials/recipes/recipes/dynamic_quantization.html

fixme: https://tutorials.pytorch.kr/recipes/recipes/dynamic_quantization.html